In [1]:
import requests
from tqdm import tqdm
import pandas as pd
pd.set_option('display.max_columns', 500)

from bs4 import BeautifulSoup
import re
import numpy as np
import os 
import urllib

from urllib.request import urlretrieve # for downloading pictures 

In [ ]:
# set to True, if you want to download photos in 'photo' folder 
DownloadPhoto = False 

# number of pages to check on website - check number of active pages on website 
EndPage = 2 # test 
# EndPage = 80

In [2]:
#Proxy list graper
# https://github.com/abdallahelsokary/Proxy-Collector-/blob/master/Proxy_Collector.py
import urllib.request
import urllib.error
import time

def proxy_list():
    try:
        time.sleep(1)
        url = "https://free-proxy-list.net/" # the source
        req = urllib.request.Request(url,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
        open_url = urllib.request.urlopen(req)
        data = open_url.read()
        soup = BeautifulSoup(data,'html.parser')
        proxy_data = soup.find_all('tr')
        Pr_list = []
        for i in proxy_data:
            Pr = "{0}:{1}".format(BeautifulSoup(str(list(i)[0]),'html.parser').text,BeautifulSoup(str(list(i)[1]),'html.parser').text)
            Pr_list.append(Pr)
        Pr_list.remove(Pr_list[0])
        Pr_list.remove(Pr_list[-1])
        
        print('Find ', len(Pr_list), 'proxies')
        
    except:
        pass
    
    return Pr_list

In [3]:
def getPage(CheckPage, proxyDict, prxList):
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    
    
    if (prxList==[]):
        prxList = proxy_list()
        prxList = list(reversed(prxList))
        prxList = prxList[0:100]
    
    r = ''
    try:
        r = requests.get(CheckPage, proxies=proxyDict, headers=header) 
    except:
        pass
    
    if (str(r)!='<Response [200]>'):
        prx = prxList.pop()
        print('Changing proxy. New one:', prx, 'Proxies left:', len(prxList))
        proxyDict = {'http': prx, 'https': prx}
        soup, r, proxyDict, prxList = getPage(CheckPage, proxyDict, prxList)
    
    data = r.text
    soup = BeautifulSoup(data, 'html.parser') 
    return soup, r, proxyDict, prxList

### ------------------------------------------------------ ### 

def getIDs(soup):
    IDs = []
    for i in soup.find_all("a"):
        if ('https://intimka.nl/Persons/' in str(i)):
            ImgIDRaw = str(i)
            ID = re.search('id=.*', str(i)).group(0).split('=')[1].split('"')[0]
            IDs.append(ID)
    return IDs

### ------------------------------------------------------ ### 

def getText(MainTable):
    Text = MainTable.find('table', {'style':"width: 440px; border: 1px solid #CCC; table-layout: fixed; overflow: hidden; margin: 0 auto 10px auto; background-color: #FFF;"})
    if (Text!=None):
        return Text.text
    else:
        return ''
    
### ------------------------------------------------------ ### 

def getPhotos(soup):
    imgs = []
    for i in soup.find_all("img"):
        if ('src="https://intimka' in str(i)):
            ImgLinkRaw = str(i)
            ImgLink = re.search('src=.*', ImgLinkRaw).group(0).split('"')[1]
            imgs.append(ImgLink)
    return imgs

### ------------------------------------------------------ ### 

def getParams(MainTable):
    Params = MainTable.tr.find('td', {'style':'vertical-align: top; padding-left: 5px;'})
    Params = Params.find('table', {'style': "width: 440px; table-layout: fixed; overflow: hidden; margin: 0 auto 10px auto; background-color: #CCC;"})
    Params = Params.tr.td.table
    
    ParamsDict = pd.read_html(str(Params))[0].dropna().set_index(0).to_dict()[1]
    for i in ParamsDict.keys():
        ParamsDict[i] = re.sub('\xa0', ' ', ParamsDict[i])
    
    return ParamsDict

### ------------------------------------------------------ ### 

def getSex(MainTable):

    Sex = MainTable.tr.find('td', {'style':'vertical-align: top; padding-left: 5px;'})
    Sex = Sex.find('table', {'style':"width: 440px; margin: 10px auto 10px auto; background-color: #CCC; border-spacing: 1px;"})
    LeftSex = Sex.tr.td.table.tr.findAll('td')[0]
    RightSex = Sex.tr.td.table.tr.findAll('td')[1]

    SDict = {'s1': 'Yes', 's3': 'No', 's2':'Comment'}

    ServiceDict = {}
    for j in LeftSex.findAll('div'):
        for i in j.findAll('a'):
            try: 
                param = re.search('>.*<', str(i)).group(0)[1:-1]
                value = str(i).split('"')[1]
                ServiceDict[param] = SDict[value]
            except:
                pass
        
    for j in RightSex.findAll('div'):
        for i in j.findAll('a'):
            try: 
                param = re.search('>.*<', str(i)).group(0)[1:-1]
                value = str(i).split('"')[1]
                ServiceDict[param] = SDict[value]
            except:
                pass
            
    return ServiceDict

### ------------------------------------------------------ ### 

def getPrice(MainTable):
    Prices = MainTable.tr.find('td', {'style':'vertical-align: top; padding-left: 5px;'}).find('table', {'cellspacing':'1', 'style':"width: 440px; margin: 10px auto 10px auto; background-color: #CCC; text-align: center; border-spacing: 1px;"})
    temp = pd.read_html(str(Prices))[0]
    
    if (temp.iloc[1][0]!='Апартаментов нет'):
        PricesDf0 = pd.DataFrame({
            'Цена': temp.iloc[1][0:3],
            'Услуга': ['Апартаменты-1 час', 'Апартаменты-2 часа', 'Апартаменты-Ночь']
        })
    else:
        PricesDf0 = pd.DataFrame({
            'Цена': [None, None, None],
            'Услуга': ['Апартаменты-1 час', 'Апартаменты-2 часа', 'Апартаменты-Ночь']
        })
    
    ####

    if (temp.iloc[2][1]!='Выезда нет'):
        
        if (temp[temp[0]=='Выезд'].shape[0]!=0):
            PricesDf1 = pd.DataFrame({
                'Цена': temp[temp[0]=='Выезд'].values[0][1:4],
                'Услуга': ['Выезд-1 час', 'Выезд-2 часа', 'Выезд-Ночь']
            })
        else:
            PricesDf1 = pd.DataFrame({
                'Цена': temp.iloc[3][1:4],
                'Услуга': ['Выезд-1 час', 'Выезд-2 часа', 'Выезд-Ночь']
            })
    else:
        PricesDf1 = pd.DataFrame({
            'Цена': [None, None, None],
            'Услуга': ['Выезд-1 час', 'Выезд-2 часа', 'Выезд-Ночь']
        }) 

    
    PricesDf = pd.concat([PricesDf0, PricesDf1], axis=0).reset_index(drop=True)
    PriceComment = temp[-1:][0].values[0] 
    if (PriceComment=='Выезд'):
        PriceComment = ''
    return dict(PricesDf.values), PriceComment 

In [4]:
prxList = []
proxyDict = {  
    'http': None,
    'https': None
}

IDsAll = []
for CurrentPage in tqdm(range(EndPage)):
    # time.sleep(3)
    CheckPage =  'http://i.intimcity.nl/persons.php?type=0&style=0&news=0&updated=0&video=N&t=&retouch=0&index='+str(CurrentPage)
    soup, r, proxyDict, prxList = getPage(CheckPage, proxyDict, prxList)
    IDsAll = IDsAll + getIDs(soup)

IDsAll = list(set(IDsAll))
print('total ids:', len(IDsAll))

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Find  311 proxies
Changing proxy. New one: 103.94.64.90:8080 Proxies left: 99


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:16<00:00,  8.22s/it]


total ids: 100


In [5]:
MainData = pd.DataFrame({}) 
PricesData = pd.DataFrame({}) 
ServicesData = pd.DataFrame({}) 
# CommentsData = pd.DataFrame({})
# StarsData = pd.DataFrame({})


for ID in tqdm(IDsAll):

    Page = 'http://i.intimcity.nl/persons.php?id=' + ID
    
    MainTable = ''
    while (MainTable==''):
        try:
            soup, r, proxyDict, prxList = getPage(Page, proxyDict, prxList)
            MainTable = soup.body.findAll('table', {'class':'hdr14'})[0].tr.find('table', {'cellspacing':'0'})
        except:
            proxyDict = {'http': None, 'https': None}
    
    imgs = getPhotos(soup)
    
    # create folder for photos and download photos 
    if (DownloadPhoto): 
        PhotoDir = './photos/'+str(ID)+'/'
        try:
            os.makedirs(PhotoDir)
        except OSError:
            pass
        for j in imgs:
            PhotoName = j.split('/')[-1]
            urlretrieve(j, PhotoDir + '/' + PhotoName)

    # parse params 
    Text = getText(MainTable)   
    t1 = getParams(MainTable)
    t2 = getSex(MainTable)
    t3, PriceComment = getPrice(MainTable)
        
    # main data 
    t1 = pd.DataFrame(t1, index=[0])
    t1['Id'] = ID
    t1['Описание'] = Text
    t1['Количество фото'] = len(imgs)
    t1['Имя'] = MainTable.findAll('h2')[0].text
    t1['Id'] = ID
    MainData = pd.concat([MainData, t1], axis=0)

    # services 
    t2 = pd.DataFrame(t2, index=[0])
    t2['Id'] = ID
    ServicesData = pd.concat([ServicesData, t2], axis=0)
        
    # prices 
    t3 = pd.DataFrame(t3, index=[0])
    t3['Id'] = ID
    t3['Комментарий к ценам'] = PriceComment
    PricesData = pd.concat([PricesData, t3], axis=0)
    
MainData = MainData.reset_index(drop=True)
PricesData = PricesData.reset_index(drop=True)
ServicesData = ServicesData.reset_index(drop=True)
# CommentsData = pd.DataFrame.reset_index(drop=True)
# StarsData = pd.DataFrame.reset_index(drop=True)

 50%|████████████████████████████████████████▌                                        | 50/100 [02:27<02:27,  2.95s/it]

Changing proxy. New one: 41.79.197.150:8080 Proxies left: 98
Changing proxy. New one: 80.211.213.200:8080 Proxies left: 97
Changing proxy. New one: 187.189.62.219:53281 Proxies left: 96
Changing proxy. New one: 24.220.198.57:8080 Proxies left: 95


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [07:04<00:00,  4.25s/it]


In [6]:
print('Main data shape:', MainData.shape)
print('Prices data shape:', PricesData.shape)
print('Services data shape:', ServicesData.shape)

Main data shape: (100, 22)
Prices data shape: (100, 8)
Services data shape: (100, 49)


In [9]:
MainData.head()

,E-mail,Id,Адрес,Вес,Возраст,Город,Грудь,Дата обновления,Имя,Интимная стрижка,Количество фото,Метро,Не моложе,Не старше,Описание,Размер обуви,Размер одежды,Район,Рейтинг,Рост,Телефон,Только для,Телефон-комментарий
0,NaN,316479,NaN,77,40,Москва,4,09.07.2018,Ms-cyrus,Полная депиляция,10,"Щукинская, Полежаевская, Войковская",18 лет (ограничение по возрасту),NaN,"Доброго времени суток, уважаемые Господа и Гос...",39,49,"СЗАО, Щукино",NaN,168,+7 (962) 315-43-04,NaN,"(инфо)Звоните, по SMS не переписываюсь."
1,NaN,323746,NaN,65,27,Москва,4,08.07.2018,Настя Flower 🌸,Полная депиляция,25,"Славянский бульвар, Деловой центр, Кутузовская",18 лет (ограничение по возрасту),NaN,"ИСПОЛНЯЮ ПОРНО МЕЧТЫ🌸\r\nАнальный секс, горлов...",36,44,"ЗАО, Фили-Давыдково",NaN,158,+7 (985) 579-29-96,NaN,"(инфо)Звоните, пишите SMS. С 13:00 до 23:00.Wh..."
2,zoloj.ole2015@yandex.ru,240266,Народного Ополчения 35,58,33,Москва,5,04.07.2018,Мила,Аккуратная стрижка,19,"Октябрьская, Третьяковская, Добрынинская",NaN,NaN,УВАЖАЕМЫЕ МУЖЧИНЫ!! ЦЕНА СКОРО БУДЕТ В 3 РАЗА ...,37,42,"ЦАО, Якиманка",NaN,178,+7 (963) 636-47-75,18-68 лет (ограничение по возрасту),"(инфо)Звоните, по SMS не переписываюсь. С 11:0..."
3,NaN,324252,NaN,48,22,Москва,2,07.07.2018,Алёна,Полная депиляция,15,"Ленинский проспект, Тульская, Академическая",18 лет (ограничение по возрасту),NaN,Сладкий МБР и ОВР ВКЛЮЧЕНЫ В СТОИМОСТЬ ЧАСА ❤️...,37,48,"ЮАО, Донской",NaN,170,+7 (968) 880-45-65,NaN,(инфо)WhatsApp
4,NaN,290608,NaN,50,25,Москва,2,19.06.2018,Марина,NaN,5,"Октябрьское поле, Беговая, Полежаевская",18 лет (ограничение по возрасту),NaN,"Горячая и раскованная, одарю теплом и лаской, ...",NaN,44,"САО, Сокол",NaN,165,+7 (963) 717-74-25,NaN,"(инфо)Звоните, по SMS не переписываюсь. С 12:0..."


In [10]:
PricesData.head()

,Апартаменты-1 час,Апартаменты-2 часа,Апартаменты-Ночь,Выезд-1 час,Выезд-2 часа,Выезд-Ночь,Id,Комментарий к ценам
0,2 000 р.,4 000 р.,10 000 р.,-,10 000 р.,20 000 р.,316479,Контактов в час: неограничено.Могу и знаю очен...
1,12 000 р.,24 000 р.,-,None,None,None,323746,Контактов в час: до 2.Экспресс не практикую.
2,4 000 р.,5 000 р.,15 000 р.,4 000 р.,5 000 р.,15 000 р.,240266,
3,8 000 р.,16 000 р.,-,-,16 000 р.,-,324252,МБР ВКЛЮЧЕНО! ОВР ВКЛЮЧЕНО!
4,3 000 р.,6 000 р.,12 000 р.,None,None,None,290608,


In [11]:
ServicesData.head()

,Cекс по телефону,Анилингус вам (мужчине),Анилингус мне (даме),Бандаж,Виртуальный секс,Госпожа,Золотой дождь вам (мужчине),Золотой дождь мне (даме),Игрушки,Игры,Клизма,Копро вам (мужчине),Куннилингус,Легкая доминация,Легкое подчинение,Лесби откровенное,Лесби-шоу легкое,Массаж классический,Массаж профессиональный,Массаж расслабляющий,Массаж урологический,Массаж эротический,Минет без резинки,Минет в машине,Минет в презервативе,Минет глубокий,Окончание в рот,Окончание на грудь,Окончание на лицо,Порка,Рабыня,Ролевые игры,Секс анальный,Секс групповой,Секс классический,Секс лесбийский,Страпон,Стриптиз не профи,Стриптиз профи,Трамплинг,Услуги семейной паре,Фетиш,Фингеринг,Фистинг анальный мне (даме),Фистинг анальный мужчине,Фистинг классический,Фото/видео съемка,Эскорт,Id
0,No,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,316479
1,No,No,Yes,Yes,No,No,No,No,Yes,Yes,Comment,No,Yes,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,Yes,No,Comment,Yes,No,No,No,Yes,Yes,Yes,Comment,Yes,Yes,No,323746
2,Comment,No,No,No,Comment,No,No,No,Comment,No,No,No,Comment,No,No,Comment,Comment,Yes,Yes,Yes,Yes,Yes,Comment,Comment,Yes,Yes,Comment,Comment,Comment,No,No,Comment,Comment,Comment,Yes,Comment,Comment,Comment,Comment,No,Comment,No,No,No,No,Comment,Comment,Comment,240266
3,No,No,Yes,Yes,No,Yes,Yes,No,Yes,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes,No,Yes,No,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Comment,Yes,No,No,Yes,No,Yes,Comment,Yes,Yes,Yes,No,Yes,No,Yes,324252
4,No,No,No,Comment,No,No,Comment,No,No,No,No,Comment,Comment,Comment,No,No,Comment,Yes,No,Yes,Comment,Yes,Comment,No,Yes,Comment,No,Comment,No,No,Comment,Comment,No,Comment,Yes,No,No,Comment,No,No,No,Comment,No,No,No,No,Comment,No,290608


In [8]:
def getPhoneComment(x):
    x = x.split(' ')[3:]
    return ' '.join(x)

def getClndPhone(x):
    x = x.split(' ')[0:3]
    return ' '.join(x)

MainData['Телефон-комментарий'] = MainData['Телефон'].apply(lambda x: getPhoneComment(x))
MainData['Телефон'] = MainData['Телефон'].apply(lambda x: getClndPhone(x))

In [ ]:
# MainData.to_csv('./data/MainData.csv',index=False)
# PricesData.to_csv('./data/PricesData.csv',index=False)
# ServicesData.to_csv('./data/ServicesData.csv',index=False)